In [1]:
# Recreating the SUNSET program from Montenbruck and Pfleger

In [3]:
import numpy as np

In [16]:
def rad(d):
    return d * np.pi / 180

def deg(a):
    return a * 180 / np.pi

def frac(x):
    return x - int(x)



pi = np.pi
pi2 = 2*pi
rad = rad(1)
deg = deg(1)
arc_second = 3600*180/pi
au = 149597870 # km
c_light = 173.14 # AU/d

In [17]:

def sin_alt(event, julian_date, hour, longitude, cphi, sphi):
    mjd = julian_date + hour/24
    t = (mjd - 51544.5)/36525
    if event == "Moon":
        ra, dec = mini_moon(t)
    else:
        ra, dec = minisun(t)
        tau = GMST(mjd) + longitude - ra
    return sphi*np.sin(dec) + cphi*np.cos(dec)*np.cos(tau)

In [18]:
# def mini_moon(t):
#     eps = 23.43929111 * rad(1)
    
#     L_0 = frac(0.606433 + 1336.855225*T) # mean longitude
#     l = 2*PI*frac(0.374897 + 1325.552410*T) # Moon's mean anomaly
#     ls = 2*PI*frac(0.993133 + 99.997361*T) # Sun's mean anomaly
#     D = 2*PI*frac(0.827361 + 1236.853086*T) # diff. long. Moon-Sun
#     F = 2*PI*frac(0.259086 + 1342.227825*T) # Dist. from ascending node
    
#     # perturbations in longitude and latitude
#     dL = 22640*np.sin(l) - 4586*np.sin(l-2*D) + 2370*np.sin(2*D) + 769*np.sin(2*l) - 668*np.sin(ls) - 412*np.sin(2*F) \
#          - 212*np.sin(2*l-2*D) - 206*np.sin(l+ls-2*D) + 192*np.sin(l+2*D) - 165*np.sin(ls-2*D) - 125*np.sin(D) \
#          - 110*np.sin(l+ls) + 148*np.sin(l-ls) - 55*np.sin(2*F-2*D)
#     S = F + (dL + 412*np.sin(2*F) + 541*np.sin(ls)) / arc_second
#     h = F - 2*D
#     N = -526*np.sin(h) + 44*np.sin(l+h) - 31*np.sin(-l+h) - 23*np.sin(ls+h)+ 11*np.sin(-ls+h) - 25*np.sin(-2*l+F) \
#         + 21*np.sin(-l+F)
    
#     # Ecliptic longitude and latitude
#     l_moon = 2*PI + frac(L_0 + dL/1296000)
#     b_moon = (18520*np.sin(S) + N) / arc_second
    
#     #e_moon = R_x(-eps) * Vec3D(Polar(l_moon, b_moon))
#     #RA = e_moon[phi]
#     #Dec = e_moon[theta]
    
#     # R_x(φ) = [ 1    0       0    ]
#     #          [ 0 +cos(φ) +sin(φ) ]
#     #          [ 0 -sin(φ) +cos(φ) ]

In [19]:
def mini_sun(t):
    eps = 23.43929111*rad
    M = 2*pi * frac(0.993133 + 99.997361*t)
    L = 2*pi * frac(0.7859453 + M/(2*pi) + (6893*np.sin(M) + 72*np.sin(2*M) + 6191.2*t) / 1296000)
    
    e_sun = 

SyntaxError: unexpected EOF while parsing (<ipython-input-19-9d348af44fa9>, line 2)

In [24]:
def mini_moon(t):
    '''
    Computes the Moon's Right-Ascension and Declination (both in radians) from the Time in Julian centuries since J2000 
    using a low-precision analytical series
    '''
    
    arc_second = 206264.8062
    
    L_0 = frac(0.606433 + 1336.855225*t) # mean longitude
    l = 2*pi*frac(0.374897 + 1325.552410*t) # Moon's mean anomaly
    ls = 2*pi*frac(0.993133 + 99.997361*t) # Sun's mean anomaly
    D = 2*pi*frac(0.827361 + 1236.853086*t) # diff. long. Moon-Sun
    F = 2*pi*frac(0.259086 + 1342.227825*t) # Dist. from ascending node
    
    # perturbations in longitude and latitude
    dL = 22640*np.sin(l) - 4586*np.sin(l-2*D) + 2370*np.sin(2*D) + 769*np.sin(2*l) - 668*np.sin(ls) - 412*np.sin(2*F) \
         - 212*np.sin(2*l-2*D) - 206*np.sin(l+ls-2*D) + 192*np.sin(l+2*D) - 165*np.sin(ls-2*D) - 125*np.sin(D) \
         - 110*np.sin(l+ls) + 148*np.sin(l-ls) - 55*np.sin(2*F-2*D)
    S = F + (dL + 412*np.sin(2*F) + 541*np.sin(ls)) / arc_second
    h = F - 2*D
    N = -526*np.sin(h) + 44*np.sin(l+h) - 31*np.sin(-l+h) - 23*np.sin(ls+h)+ 11*np.sin(-ls+h) - 25*np.sin(-2*l+F) \
        + 21*np.sin(-l+F)
    
    # Ecliptic longitude and latitude
    l_moon = 2*pi + frac(L_0 + dL/1296000)
    b_moon = (18520*np.sin(S) + N) / arc_second
    
    # Equatorial coordinates
    CB = np.cos(b_moon)
    x = CB * np.cos(l_moon)
    v = CB * np.sin(l_moon)
    w = np.sin(b_moon)
    y = np.dot([0.91748, -0.39778], [v, w])
    z = np.dot([0.39778, 0.91748], [v, w])
    rho = np.sqrt(1 - z**2)
    dec = deg * np.arctan(z/rho)
    ra = (24/pi) * np.arctan(y / (x + rho))
    if ra < 0: ra += 24
    return ra, dec

In [25]:
mini_moon(0)

(2.0956870292997443, 18.23660699640844)

In [ ]:
def find_events(event, mjd0h, longitude, latitude):
    sin_h0 = {
        'moonrise':np.sin(rad(8/60)),
        'sunrise': np.sin(rad(-50/60)),
        'civil twilight': np.sin(rad(-6)),
        'nautical twilight': np.sin(rad(-12)),
        'astronomical twilight': np.sin(rad(-18)),
    }
    
    Cphi = np.cos(latitude)
    Sphi = np.sin(latitude)
    
    # Initialize for search
    y_minus = sin_alt(event, mjd0h, 0, longitude, Cphi, Sphi) - sin_h0[event]
    above = (y_minus>0.0)
    rises = false
    sets = false
    for hour in range(1,24,2):
        y_0 = sin_alt(event, mjd0h, hour, longitude, Cphi, Sphi) - sin_h0[event]
        y_plus = sin_alt(event, mjd0h, hour+1, longitude, Cphi, Sphi) - sin_h0[event]
        xe, ye, root1, root2, n_root = quad(y_minus, y_0, y_plus)
        if n_root == 1:
            if y_minus < 0:
                lt_rise = hour + root1
                rises = True
            else:
                lt_set = hour + root1
                sets = True
        if n_root == 2:
            if ye < 0:
                lt_rise = hour+root2
                lt_set = hour+root1
            else:
                lt_rise = hour+root1
                lt_set = hour+root2
            rises = True
            sets = True
        y_minus = y_plus
        
